# Data Cleaning and Processing

## Required Libraries

In [1]:
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime
import pdb
import re
import os
import time
import datetime
from pathlib import Path
import pickle
import dateutil
from dateutil.relativedelta import relativedelta
import random
from os import system
from math import floor
from copy import deepcopy
from dask import dataframe as dd

## NYT Data

### Loading in Best Seller list

In [2]:
best_sellers = pd.read_pickle('./revised_df.pkl','zip')

In [3]:
best_sellers.head()

,Title,List_Type,Update_Interval,Author,Rank,Isbn10,Isbn13,BestSeller_Date,List_Published_Date,Publisher,Weeks_on_List,Amazon_Link,Apple_Books_Link,Barnes_and_Nobel_Link,Books_A_Million_Link,Bookshop_Link,IndieBound_Link
0,ODD HOURS,Hardcover Fiction,WEEKLY,Dean R Koontz,1,0553807056,9780553807059,2008-05-24,2008-06-08,Bantam,1,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780553807059,https://www.indiebound.org/book/9780553807059?...
1,THE HOST,Hardcover Fiction,WEEKLY,Stephenie Meyer,2,0316068047,9780316068048,2008-05-24,2008-06-08,"Little, Brown",3,http://www.amazon.com/The-Host-Novel-Stephenie...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780316068048,https://www.indiebound.org/book/9780316068048?...
2,LOVE THE ONE YOU'RE WITH,Hardcover Fiction,WEEKLY,Emily Giffin,3,0312348673,9780312348670,2008-05-24,2008-06-08,St. Martin's,2,http://www.amazon.com/Love-Youre-With-Emily-Gi...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780312348670,https://www.indiebound.org/book/9780312348670?...
3,THE FRONT,Hardcover Fiction,WEEKLY,Patricia Cornwell,4,0399154183,9780399154188,2008-05-24,2008-06-08,Putnam,1,http://www.amazon.com/The-Front-Garano-Patrici...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780399154188,https://www.indiebound.org/book/9780399154188?...
4,SNUFF,Hardcover Fiction,WEEKLY,Chuck Palahniuk,5,0385517882,9780385517881,2008-05-24,2008-06-08,Doubleday,1,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780385517881,https://www.indiebound.org/book/9780385517881?...


In [4]:
len(best_sellers)

42791

### Finding all unique Books

In [5]:
uni = best_sellers.Isbn13.unique()

In [6]:
len(uni)

6761

In [7]:
removed_dups = best_sellers.drop_duplicates(keep='last',subset='Isbn13')

In [8]:
len(removed_dups)

6761

In [9]:
removed_dups.head()

,Title,List_Type,Update_Interval,Author,Rank,Isbn10,Isbn13,BestSeller_Date,List_Published_Date,Publisher,Weeks_on_List,Amazon_Link,Apple_Books_Link,Barnes_and_Nobel_Link,Books_A_Million_Link,Bookshop_Link,IndieBound_Link
3,THE FRONT,Hardcover Fiction,WEEKLY,Patricia Cornwell,4,0399154183,9780399154188,2008-05-24,2008-06-08,Putnam,1,http://www.amazon.com/The-Front-Garano-Patrici...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780399154188,https://www.indiebound.org/book/9780399154188?...
4,SNUFF,Hardcover Fiction,WEEKLY,Chuck Palahniuk,5,0385517882,9780385517881,2008-05-24,2008-06-08,Doubleday,1,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780385517881,https://www.indiebound.org/book/9780385517881?...
9,THE REVOLUTION,Hardcover Nonfiction,WEEKLY,Ron Paul,5,0446537519,9780446537513,2008-05-24,2008-06-08,Grand Central,5,http://www.amazon.com/The-Revolution-Manifesto...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780446537513,https://www.indiebound.org/book/9780446537513?...
16,THE GOOD GUY,Mass Market Paperback,WEEKLY,Dean R Koontz,2,0553589113,9780553589115,2008-05-24,2008-06-08,Bantam,4,http://www.amazon.com/Warriors-Revenge-Exectio...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780553589115,https://www.indiebound.org/book/9780553589115?...
17,INVISIBLE PREY,Mass Market Paperback,WEEKLY,John Sandford,3,0425221156,9780425221150,2008-05-24,2008-06-08,Berkley,5,http://www.amazon.com/Invisible-Prey-John-Sand...,https://du-gae-books-dot-nyt-du-prd.appspot.co...,https://www.anrdoezrs.net/click-7990613-118195...,https://www.anrdoezrs.net/click-7990613-35140?...,https://bookshop.org/a/3546/9780425221150,https://www.indiebound.org/book/9780425221150?...


### Turning Unique Titiles into list to filter reviews

In [10]:
filtered = list(removed_dups['Title'])
filtered

['THE FRONT',
 'SNUFF',
 'THE REVOLUTION',
 'THE GOOD GUY',
 'INVISIBLE PREY',
 'THE BOURNE BETRAYAL',
 'NICK OF TIME',
 'THE CARLYLES',
 'BROADWAY BARKS',
 'SUNDAYS AT TIFFANY’S',
 'STOLEN INNOCENCE',
 'THE LOST DUKE OF WYNDHAM',
 'HIDE',
 'HUNGRY GIRL',
 'TWEAK',
 'MASSIE',
 'WARRIORS: POWER OF THREE',
 'BLOOD NOIR',
 'AUDITION',
 'PILLARS OF THE EARTH',
 'LETTERS TO A YOUNG SISTER',
 'INDIANA JONES AND THE KINGDOM OF THE CRYSTAL SKULL',
 'DIRT ON MY SHIRT',
 'PRETTY LITTLE LIARS',
 'NOTHING TO LOSE',
 'PLAGUE SHIP',
 'ALWAYS BY MY SIDE',
 'THE POST-AMERICAN WORLD: RELEASE 2.0',
 'LOVER ENSHRINED',
 'THE HOLLOW',
 "BOBBY FLAY'S GRILL IT!",
 'BEST AT EVERYTHING',
 'CHASING HARRY WINSTON',
 "LOVE THE ONE YOU'RE WITH",
 'HIGH NOON',
 'THE JUDAS STRAIN',
 'RETURN TO SUMMERHOUSE',
 'QUANTUM WELLNESS',
 'READ ALL ABOUT IT!',
 'LADYBUG GIRL',
 'DON’T BUMP THE GLUMP!',
 'ROGUE',
 'FAST TRACK',
 'END OF DAYS',
 'JUNIE B. JONES',
 'SAIL',
 'TAILSPIN',
 'THE MONSTER OF FLORENCE',
 'THE FRIDAY N

## Review Data

### Loading Amazon Review Data

In [11]:
path = '../../../Downloads/'                               #<---local directory path where dataset resides
filename = 'amazon_reviews_us_Books_v1_00' #<---name of dataset being processed
ext = '.tsv'                                 #<---file extension (tab seperated values)

cols = ['marketplace',       #<---2 letter country code of review marketplace
        'customer_id',       #<---random identifier to aggregate reviews by single author
        'review_id',         #<---unique ID of review
        'product_id',        #<---unique ID of product to which review pertains
        'product_parent',    #<---random identifier to aggregate reviews for same product
        'product_title',     #<---product title
        'product_category',  #<---product category to group dataset into coherent parts 
        'star_rating',       #<---1-5 star rating of product
        'helpful_votes',     #<---number of helpful votes review received
        'total_votes',       #<---total number of votes review received
        'vine',              #<---review part of Vine program
        'verified_purchase', #<---review of verified purchase
        'review_headline',   #<---review title
        'review_body',       #<---review text
        'review_date']       #<---review date

chunksize = 10 ** 6
start = time.time()
dask_df = dd.read_csv(path+filename+ext,sep='\t',usecols = cols,dtype={'helpful_votes': 'float64','star_rating': 'object','total_votes': 'float64'})
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
# df = pd.read_csv(path+filename+ext,
#                  sep='\t',
#                  usecols = cols,
#                 low_memory=False)


Read csv with dask:  0.04118919372558594 sec


In [12]:
dask_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,25933450,RJOVP071AVAJO,0439873800,84656342,There Was an Old Lady Who Swallowed a Shell!,Books,5,0.0,0.0,N,Y,Five Stars,I love it and so does my students!,2015-08-31
1,US,1801372,R1ORGBETCDW3AI,1623953553,729938122,I Saw a Friend,Books,5,0.0,0.0,N,Y,"Please buy ""I Saw a Friend""! Your children wil...",My wife and I ordered 2 books and gave them as...,2015-08-31
2,US,5782091,R7TNRFQAOUTX5,142151981X,678139048,"Black Lagoon, Vol. 6",Books,5,0.0,0.0,N,Y,Shipped fast.,Great book just like all the others in the ser...,2015-08-31
3,US,32715830,R2GANXKDIFZ6OI,014241543X,712432151,If I Stay,Books,5,0.0,0.0,N,N,Five Stars,So beautiful,2015-08-31
4,US,14005703,R2NYB6C3R8LVN6,1604600527,800572372,Stars 'N Strips Forever,Books,5,2.0,2.0,N,Y,Five Stars,Enjoyed the author's story and his quilts are ...,2015-08-31


### Cutting Data to only Verified Purchases

In [13]:
verified_df = dask_df[dask_df['verified_purchase'] == 'Y']

In [77]:
print(len(verified_df))

7388560


In [2]:
#verified_df.tail(10)

### Removing Unnecessary Columns

In [15]:
columns = ['customer_id','review_id','product_id','product_parent','product_category','vine',]

In [16]:
slim_verified = verified_df.drop(columns=columns)

In [3]:
#slim_verified.head()

### Making Titles uniform

In [18]:
slim_verified['product_title'] = slim_verified['product_title'].str.upper()

In [4]:
#slim_verified.head()

### Resetting Index

In [20]:
slim_verified.reset_index(drop=True)

,marketplace,product_title,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date
npartitions=105,,,,,,,,,
,object,object,object,float64,float64,object,object,object,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [5]:
#slim_verified.head()

### Getting reviews for only possitive class

In [22]:
series = slim_verified.product_title.isin(filtered)
filtered_df = slim_verified[series]

In [64]:
len(filtered_df)

153760

### Changing back to Pandas dataframe

In [23]:
filtered_df = filtered_df.compute()

In [24]:
type(filtered_df)

pandas.core.frame.DataFrame

### Saving Data

In [26]:
filtered_df.to_csv(r'./possitive_case.csv')


## Getting Negative Class reviews

In [27]:
inverse_series = ~slim_verified.product_title.isin(filtered)
inverse_df = slim_verified[inverse_series]


In [28]:
len(inverse_df)

7234800

In [29]:
type(inverse_df)

dask.dataframe.core.DataFrame

In [30]:
inverse_df.repartition(npartitions=inverse_df.npartitions)

,marketplace,product_title,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date
npartitions=105,,,,,,,,,
,object,object,object,float64,float64,object,object,object,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [31]:
len(inverse_df)

7234800

In [32]:
neg_df = inverse_df.repartition(npartitions=inverse_df.npartitions)

In [33]:
len(neg_df)

7234800

In [34]:
inverse_df['Status'] = 0

In [6]:
#inverse_df.head()

In [37]:
inverse_df.npartitions

105

### Cutting the data down to match Possitive class

In [95]:
neg_df = inverse_df.partitions[::59]

In [96]:
len(neg_df)

157551

In [97]:
neg_df.npartitions

2

In [7]:
#neg_df.head()

In [99]:
neg_df = neg_df.reset_index(drop=True)

In [8]:
#neg_df.head()

In [101]:
neg_df = neg_df.compute()

In [102]:
type(neg_df)

pandas.core.frame.DataFrame

### Saving Data

In [103]:
neg_df.to_csv(r'./negative_case.csv')